In [1]:
import pandas as pd
import glob
import os

# Directory path
csv_dir = (
    "/scratch/project/tcr_ml/gnn_release/icantcrscoring/model_2025_sc_curated/seekgene"
)

# Get all CSV files in the directory
csv_files = [
    f
    for f in glob.glob(os.path.join(csv_dir, "*.csv"))
    if os.path.basename(f) != "summary.csv"
]
# Load all CSV files into a list of DataFrames
dataframes = []
for file in csv_files:
    df = pd.read_csv(file)
    # Optional: add a column to track which file the data came from
    df["source_file"] = os.path.basename(file)
    dataframes.append(df)

# If you want to combine all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

combined_df

,AA_seq,CloneFreq,prob,high,transformed_score,source_file
0,CASASDGAGDQPQHF,0.037352,0.920785,True,3.439338e-02,S2_merged.csv
1,CASSRAPASTEAFF,0.037352,0.423499,False,1.581865e-02,S2_merged.csv
2,CASSPGQGELFF,0.011348,0.256990,False,2.916200e-03,S2_merged.csv
3,CSARDRGGENTGELFF,0.008983,0.991506,True,8.907149e-03,S2_merged.csv
4,CSVPGTDYNEQFF,0.008983,0.004697,False,4.219959e-05,S2_merged.csv
...,...,...,...,...,...,...
5629,CSAPGTQIYEQYF,0.000386,0.251051,False,9.693102e-05,S3_merged.csv
5630,CASSLQVIGTDTQYF,0.000386,0.384414,False,1.484223e-04,S3_merged.csv
5631,CASSLTGLNTEAFF,0.000386,0.000260,False,1.004468e-07,S3_merged.csv
5632,CASSFQGTVYEQYF,0.000386,0.529522,False,2.044488e-04,S3_merged.csv


In [2]:
# Directory path
csv_dir = (
    "/scratch/project/tcr_ml/gnn_release/icantcrscoring/model_2025_sc_curated/PICA"
)

# Get all CSV files in the directory
csv_files = [
    f
    for f in glob.glob(os.path.join(csv_dir, "*.csv"))
    if os.path.basename(f) != "summary.csv"
]
# Load all CSV files into a list of DataFrames
dataframes = []
for file in csv_files:
    df = pd.read_csv(file)
    # Optional: add a column to track which file the data came from
    df["source_file"] = os.path.basename(file)
    dataframes.append(df)

# If you want to combine all DataFrames into one
combined_control_df = pd.concat(dataframes, ignore_index=True)

combined_control_df

,AA_seq,CloneFreq,prob,high,transformed_score,source_file
0,CASTPGDEQYF,0.003607,0.575344,False,0.002075,20240918_WGS_20240924_sc_PICA0008-PICA0032_Poo...
1,CASSLGSGQYF,0.001945,0.588268,False,0.001144,20240918_WGS_20240924_sc_PICA0008-PICA0032_Poo...
2,CASSLGSGQYF,0.001945,0.588268,False,0.001144,20240918_WGS_20240924_sc_PICA0008-PICA0032_Poo...
3,CASSLGSGQYF,0.001945,0.588268,False,0.001144,20240918_WGS_20240924_sc_PICA0008-PICA0032_Poo...
4,CASSLGSGQYF,0.001945,0.588268,False,0.001144,20240918_WGS_20240924_sc_PICA0008-PICA0032_Poo...
...,...,...,...,...,...,...
149200,CATSDSTRGDSYNEQFF,0.000038,0.144616,False,0.000005,20241106_WGS_20241106_sc_PICA0033-PICA0069_Poo...
149201,CSAGGDRSNQPQHF,0.000038,0.086664,False,0.000003,20241106_WGS_20241106_sc_PICA0033-PICA0069_Poo...
149202,CSAREPGSSPLHF,0.000038,0.909164,True,0.000035,20241106_WGS_20241106_sc_PICA0033-PICA0069_Poo...
149203,CAISETSEVGYEQYF,0.000038,0.329552,False,0.000013,20241106_WGS_20241106_sc_PICA0033-PICA0069_Poo...


In [ ]:
# %%
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --------- assumes combined_df and combined_control_df already exist ----------


# Reuse your utilities
def fraction_to_percentile(
    x, weights=None, method="hazen", open_interval=False, eps=1e-9, nan_policy="omit"
):
    x = np.asarray(x, dtype=float)
    n = x.size
    if n == 0:
        return x
    has_nan = np.isnan(x).any()
    if has_nan:
        if nan_policy == "raise":
            raise ValueError("NaNs present with nan_policy='raise'.")
        if nan_policy == "propagate":
            return np.full_like(x, np.nan, dtype=float)
    if weights is None:
        w = np.ones_like(x, dtype=float)
    else:
        w = np.asarray(weights, dtype=float)
        if w.shape != x.shape:
            raise ValueError("weights must have same shape as x")
        if np.any(w < 0):
            raise ValueError("weights must be nonnegative")

    valid = ~np.isnan(x)
    xv = x[valid]
    wv = w[valid]
    m = xv.size
    out = np.full_like(x, np.nan, dtype=float)
    if m == 0:
        return out
    if m == 1:
        out[valid] = 0.5
        return out

    order = np.argsort(xv, kind="mergesort")
    xv_sorted = xv[order]
    wv_sorted = wv[order]

    def _midranks_for_ties(sorted_vals_len: int, diffs: np.ndarray):
        m = sorted_vals_len
        if m == 0:
            return np.empty(0, dtype=float)
        boundaries = np.flatnonzero(diffs != 0.0)
        starts = np.r_[0, boundaries + 1]
        stops = np.r_[boundaries, m - 1]
        ranks = np.empty(m, dtype=float)
        for s, e in zip(starts, stops):
            mid = (s + e) / 2.0
            ranks[s : e + 1] = mid + 1.0
        return ranks

    def _plotting_position(ranks, m, method: str):
        if method == "weibull":
            return ranks / (m + 1.0)
        if method == "hazen":
            return (ranks - 0.5) / m
        if method == "blom":
            return (ranks - 0.375) / (m + 0.25)
        if method == "bernard":
            return (ranks - 3.0 / 8.0) / (m + 0.25)
        if method == "rank":
            return np.full(m, 0.5) if m == 1 else (ranks - 1.0) / (m - 1.0)
        raise ValueError(f"Unknown method '{method}'")

    if weights is None or np.allclose(wv_sorted, 1.0):
        diffs = np.diff(xv_sorted)
        ranks = _midranks_for_ties(len(xv_sorted), diffs)
        p_sorted = _plotting_position(ranks, m, method)
    else:
        uniq_vals, idx_start, counts = np.unique(
            xv_sorted, return_index=True, return_counts=True
        )
        group_weights = np.add.reduceat(wv_sorted, idx_start)
        cw = np.cumsum(group_weights)
        total = cw[-1]
        cw_prev = cw - group_weights
        p_group = (cw_prev + 0.5 * group_weights) / total
        p_sorted = np.repeat(p_group, counts)

    inv = np.empty(m, dtype=int)
    inv[order] = np.arange(m)
    out[valid] = p_sorted[inv]
    if open_interval:
        out[out <= 0.0] = eps
        out[out >= 1.0] = 1.0 - eps
    return out


def combined_score_sample_capped(
    P,
    F_raw,
    t_P=0.5,
    t_R=0.5,
    alpha=0.6,
    beta=0.9,
    delta_max=0.25,
    floor=0.01,
    ceil=0.99,
):
    P = np.clip(np.asarray(P, dtype=float), 0.0, 1.0)
    R = fraction_to_percentile(F_raw)

    mask_high = (P > t_P) & (R > t_R)
    mask_low = (P < t_P) & (R < t_R)

    A_high = np.where(mask_high, np.minimum(P, R), 0.0)
    A_low = np.where(mask_low, np.minimum(1.0 - P, 1.0 - R), 0.0)

    delta = alpha * A_high - beta * A_low
    delta = np.clip(delta, -delta_max, delta_max)

    S = np.clip(P + delta, floor, ceil)
    return S, R


def summarize_probs(x):
    x = np.clip(np.asarray(x, dtype=float), 1e-9, 1 - 1e-9)
    logits = np.log(x / (1 - x))
    return pd.Series(
        {
            "arithmetic_mean": np.mean(x),
            "median": np.median(x),
            "geometric_mean": np.exp(np.mean(np.log(x))),
            "inv_logit_mean": 1 / (1 + np.exp(-np.mean(logits))),
        }
    )


def build_metric_table(df):
    # metrics over score_adj per source file
    sm = df.groupby("source_file")["score_adj"].apply(summarize_probs).reset_index()

    # reshape to long using a temp value_name that does not collide
    sm_long = sm.melt(
        id_vars="source_file", var_name="level_1", value_name="metric_value"
    )

    # baseline mean of original prob for comparison
    old_means = (
        df.groupby("source_file")["prob"]
        .mean()
        .rename("metric_value")  # match temp name
        .reset_index()
    )
    old_means["level_1"] = "old_arithmetic_mean"

    # combine and standardize column name
    out = pd.concat(
        [
            sm_long[["source_file", "level_1", "metric_value"]],
            old_means[["source_file", "level_1", "metric_value"]],
        ],
        ignore_index=True,
    ).rename(columns={"metric_value": "score_adj"})

    return out


def roc_curve_manual(y_true: np.ndarray, y_score: np.ndarray):
    y_true = y_true.astype(int)
    order = np.argsort(-y_score)
    y_true_sorted = y_true[order]
    y_score_sorted = y_score[order]
    P = y_true_sorted.sum()
    N = len(y_true_sorted) - P
    if P == 0 or N == 0:
        return np.array([0.0, 1.0]), np.array([0.0, 1.0]), np.nan
    tps = np.cumsum(y_true_sorted)
    fps = np.cumsum(1 - y_true_sorted)
    diffs = np.diff(y_score_sorted)
    idx = np.where(diffs != 0)[0]
    idx = np.r_[idx, len(y_true_sorted) - 1]
    tpr = tps[idx] / P
    fpr = fps[idx] / N
    tpr = np.r_[0.0, tpr]
    fpr = np.r_[0.0, fpr]
    auc = np.trapz(tpr, fpr)
    return fpr, tpr, auc


def compute_auc_for_metrics(df_cancer_long, df_control_long, metrics):
    # combine and compute AUC for each metric
    dfc = df_cancer_long.copy()
    dfc["group"] = "cancer"
    dfk = df_control_long.copy()
    dfk["group"] = "control"
    df_long = pd.concat([dfc, dfk], ignore_index=True)
    out = {}
    for m in metrics:
        sub = df_long[df_long["level_1"] == m].dropna(subset=["score_adj", "group"])
        if sub.empty:
            out[m] = np.nan
            continue
        y_true = (sub["group"].values == "cancer").astype(int)
        y_score = sub["score_adj"].astype(float).values
        _, _, auc = roc_curve_manual(y_true, y_score)
        out[m] = auc
    return out


# Define the grid
alpha_grid = np.linspace(0.0, 8.0, 33)  # step 0.25
beta_grid = np.r_[
    np.logspace(-4, -1, 7), np.linspace(0.2, 1.0, 9)
]  # 1e-4..0.1 log, then 0.2..1.0 linear

# Choose which metrics to optimize
metrics_all = [
    "old_arithmetic_mean",
    "arithmetic_mean",
    "median",
    "geometric_mean",
    "inv_logit_mean",
]
metrics_all = [
    m
    for m in metrics_all
    if m
    in set(
        [
            "old_arithmetic_mean",
            "arithmetic_mean",
            "median",
            "geometric_mean",
            "inv_logit_mean",
        ]
    )
]

# Storage
auc_cube = {m: np.full((len(alpha_grid), len(beta_grid)), np.nan) for m in metrics_all}

# Pre-extract base arrays to avoid repeated Series access cost
P_cancer = combined_df["prob"].to_numpy(float)
F_cancer = combined_df["CloneFreq"].to_numpy(float)
P_control = combined_control_df["prob"].to_numpy(float)
F_control = combined_control_df["CloneFreq"].to_numpy(float)

# Fixed thresholds and clipping as in your code
kw = dict(t_P=0.5, t_R=0.5, delta_max=0.25, floor=0.01, ceil=0.99)

# Grid search
for i, a in enumerate(alpha_grid):
    for j, b in enumerate(beta_grid):
        # adjust scores
        S_c, R_c = combined_score_sample_capped(
            P=P_cancer, F_raw=F_cancer, alpha=a, beta=b, **kw
        )
        S_k, R_k = combined_score_sample_capped(
            P=P_control, F_raw=F_control, alpha=a, beta=b, **kw
        )

        # build temp dataframes
        tmp_cancer = combined_df[["source_file", "prob"]].copy()
        tmp_control = combined_control_df[["source_file", "prob"]].copy()
        tmp_cancer["score_adj"] = S_c
        tmp_control["score_adj"] = S_k

        # build per-metric tables
        sm_c = build_metric_table(tmp_cancer)
        sm_k = build_metric_table(tmp_control)

        # compute auc per metric
        aucs = compute_auc_for_metrics(sm_c, sm_k, metrics_all)
        for m in metrics_all:
            auc_cube[m][i, j] = aucs[m]

# Find the best alpha and beta for each metric
best_rows = []
for m in metrics_all:
    mat = auc_cube[m]
    idx = np.nanargmax(mat)
    i, j = np.unravel_index(idx, mat.shape)
    best_rows.append(
        {
            "metric": m,
            "best_alpha": float(alpha_grid[i]),
            "best_beta": float(beta_grid[j]),
            "best_auc": float(mat[i, j]),
        }
    )

best_df = pd.DataFrame(best_rows).sort_values("best_auc", ascending=False)
print("Best alpha and beta per metric based on AUROC\n")
print(best_df.to_string(index=False))

# Plot heatmaps per metric
n_metrics = len(metrics_all)
ncols = 2
nrows = int(np.ceil(n_metrics / ncols))
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 4 * nrows))

axes = np.array(axes).reshape(-1)
for k, m in enumerate(metrics_all):
    ax = axes[k]
    Z = auc_cube[m]
    im = ax.imshow(
        Z,
        origin="lower",
        aspect="auto",
        extent=[beta_grid.min(), beta_grid.max(), alpha_grid.min(), alpha_grid.max()],
    )
    ax.set_title(f"AUROC heatmap - {m}")
    ax.set_xlabel("beta")
    ax.set_ylabel("alpha")
    # mark best point
    idx = np.nanargmax(Z)
    i, j = np.unravel_index(idx, Z.shape)
    ax.plot(beta_grid[j], alpha_grid[i], marker="o", markersize=6)
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

# remove empty subplots if any
for q in range(k + 1, len(axes)):
    fig.delaxes(axes[q])

plt.tight_layout()
plt.show()

# Optional: line plot of best AUROC by metric
plt.figure(figsize=(8, 4))
plt.plot(best_df["metric"], best_df["best_auc"], marker="o")
plt.title("Best AUROC by metric")
plt.xlabel("Metric")
plt.ylabel("Best AUROC")
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()

ValueError: Length of values (3) does not match length of index (12)